In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
from skimage.transform import warp, AffineTransform

gesture_folders = ['six', 'seven', 'eight', 'nine', 'ten']

for folder in gesture_folders:
    folder_path = f'/content/drive/My Drive/Colab Notebooks/hand_gesture/{folder}'
    output_folder_path = f'/content/drive/My Drive/Colab Notebooks/hand_gesture/{folder}'

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    def augment_image(image):
        augmented_images = []

        # 좌우로 15도, 30도 회전
        for angle in [-15, 15, -30, 30]:
            rows, cols = image.shape[:2]
            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
            rotated_image = cv2.warpAffine(image, M, (cols, rows))
            augmented_images.append(rotated_image)

        # 뒤집은 이미지
        flipped_image = cv2.flip(image, 1)
        augmented_images.append(flipped_image)

        # 뒤집은 이미지에 회전 추가
        for angle in [-15, 15, -30, 30]:
            rows, cols = flipped_image.shape[:2]
            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
            rotated_flipped_image = cv2.warpAffine(flipped_image, M, (cols, rows))
            augmented_images.append(rotated_flipped_image)

        # 확대
        zoom_factor = 1.1
        zoomed_image = cv2.resize(image, (0, 0), fx=zoom_factor, fy=zoom_factor)
        augmented_images.append(zoomed_image)

        # 뒤집은 이미지 확대
        flipped_zoomed_image = cv2.resize(flipped_image, (0, 0), fx=zoom_factor, fy=zoom_factor)
        augmented_images.append(flipped_zoomed_image)

        return augmented_images

    for i in range(1, 11):
        filename = f'{folder}_{i}.jpg'
        image_path = os.path.join(folder_path, filename)

        original_image = cv2.imread(image_path)

        augmented_images = augment_image(original_image)

        for j, aug_image in enumerate(augmented_images):
            output_filename = f'{folder}_{i}_{j+1}.jpg'
            output_path = os.path.join(output_folder_path, output_filename)

            cv2.imwrite(output_path, aug_image, [int(cv2.IMWRITE_JPEG_QUALITY), 95])


KeyboardInterrupt: 

In [2]:
import os
import numpy as np
from PIL import Image

gesture_folders = ['six', 'seven', 'eight', 'nine', 'ten']

image_size = (224, 224)

X = []
y = []

for folder in gesture_folders:
    image_dir = f'/content/drive/My Drive/Colab Notebooks/hand_gesture/{folder}'

    label = gesture_folders.index(folder)

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(image_dir, filename)

            image = Image.open(image_path).resize(image_size)

            image_array = np.array(image)

            X.append(image_array)
            y.append(label)

X = np.array(X)
y = np.array(y)

print(f'이미지: {X.shape}')
print(f'라벨: {y.shape}')


이미지: (600, 224, 224, 3)
라벨: (600,)


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X = np.array(X)
y = np.array(y)

unique_labels = np.unique(y)

X_train = []
y_train = []
X_val = []
y_val = []
X_test = []
y_test = []

for label in unique_labels:
    label_idx = np.where(y == label)[0]
    X_label = X[label_idx]
    y_label = y[label_idx]

    X_label_train, X_label_test, y_label_train, y_label_test = train_test_split(X_label, y_label, test_size=0.2, random_state=77)

    X_label_train, X_label_val, y_label_train, y_label_val = train_test_split(X_label_train, y_label_train, test_size=0.2, random_state=77)

    X_train.extend(X_label_train)
    y_train.extend(y_label_train)
    X_val.extend(X_label_val)
    y_val.extend(y_label_val)
    X_test.extend(X_label_test)
    y_test.extend(y_label_test)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

y_train = to_categorical(y_train, num_classes=5)
y_val = to_categorical(y_val, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

print(f'학습 데이터 : {X_train.shape}, {y_train.shape}')
print(f'검증 데이터 : {X_val.shape}, {y_val.shape}')
print(f'테스트 데이터 : {X_test.shape}, {y_test.shape}')


학습 데이터 : (380, 224, 224, 3), (380, 5)
검증 데이터 : (100, 224, 224, 3), (100, 5)
테스트 데이터 : (120, 224, 224, 3), (120, 5)


In [6]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')


# VGG16 모델
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
output = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1)

model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=100,
          validation_data=(X_val, y_val),
          callbacks=[early_stopping, reduce_lr])

# 테스트
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


Epoch 1/100
12/12 [==============================] - 24s 1s/step - loss: 1.7093 - accuracy: 0.3658 - val_loss: 1.8074 - val_accuracy: 0.3800 - lr: 0.0010
Epoch 2/100
12/12 [==============================] - 5s 434ms/step - loss: 1.0085 - accuracy: 0.5895 - val_loss: 1.4556 - val_accuracy: 0.5200 - lr: 0.0010
Epoch 3/100
12/12 [==============================] - 5s 406ms/step - loss: 0.6546 - accuracy: 0.7684 - val_loss: 1.1333 - val_accuracy: 0.6700 - lr: 0.0010
Epoch 4/100
12/12 [==============================] - 5s 427ms/step - loss: 0.4659 - accuracy: 0.8184 - val_loss: 0.8238 - val_accuracy: 0.7000 - lr: 0.0010
Epoch 5/100
12/12 [==============================] - 5s 406ms/step - loss: 0.4269 - accuracy: 0.8421 - val_loss: 0.6750 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 6/100
12/12 [==============================] - 7s 581ms/step - loss: 0.3048 - accuracy: 0.8947 - val_loss: 0.4755 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 7/100
12/12 [==============================] - 5s 433ms/step

In [7]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# ResNet50 모델
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1)
checkpoint = ModelCheckpoint('resnet50_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True)

model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# 테스트
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


94765736/94765736 [==============================] - 3s 0us/step


Epoch 1/100
12/12 [==============================] - ETA: 0s - loss: 48.2545 - accuracy: 0.2974
Epoch 1: val_accuracy improved from -inf to 0.33000, saving model to resnet50_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - 21s 1s/step - loss: 48.2545 - accuracy: 0.2974 - val_loss: 19.7069 - val_accuracy: 0.3300 - lr: 0.0010
Epoch 2/100
12/12 [==============================] - ETA: 0s - loss: 14.9658 - accuracy: 0.4974
Epoch 2: val_accuracy improved from 0.33000 to 0.69000, saving model to resnet50_model.h5
12/12 [==============================] - 13s 1s/step - loss: 14.9658 - accuracy: 0.4974 - val_loss: 2.8527 - val_accuracy: 0.6900 - lr: 0.0010
Epoch 3/100
12/12 [==============================] - ETA: 0s - loss: 6.0507 - accuracy: 0.5421
Epoch 3: val_accuracy improved from 0.69000 to 0.73000, saving model to resnet50_model.h5
12/12 [==============================] - 10s 867ms/step - loss: 6.0507 - accuracy: 0.5421 - val_loss: 1.4912 - val_accuracy: 0.7300 - lr: 0.0010
Epoch 4/100
12/12 [==============================] - ETA: 0s - loss: 3.4630 - accuracy: 0.5711
Epoch 4: val_accuracy improved from 0.73000 to 0.77000, saving model to resnet50_model.h5
12/12 [=====